In [ ]:
#load in file
filepath = 'facebook-messages\messages\inbox\MirrenWilson_DIFAxVcXfw\message_1.json'

In [ ]:
import json
with open(filepath) as file:
    chat_history = json.load(file)

In [ ]:
#Reading through the messages
chat_history

In [ ]:
#looking at keys and exploring data
chat_history.keys()

In [ ]:
chat_history['messages']

In [ ]:
import pandas as pd

In [ ]:
#put the data into a pandas dataframe
messages = pd.DataFrame(chat_history['messages'])

In [ ]:
#Convert the facebook millisecond timestamp to a useable date format
def convertion(timestamp):
    return pd.to_datetime(timestamp,unit='ms')

In [ ]:
convertion(1559935327750)

In [ ]:
#apply the convertion to the dataframe 
messages['date'] = messages['timestamp_ms'].apply(convertion)

In [ ]:
messages

In [ ]:
#defining and applying more date types

def month(date):
    return date.month
def year(date):
    return date.year
def day(date):
    return date.day
def daysinmonth(date):
    return date.dayofyear	

In [ ]:
messages['month'] = messages['date'].apply(month)
messages['year'] = messages['date'].apply(year)
messages['day'] = messages['date'].apply(day)
messages['date2'] = messages['date'].apply(daysinmonth)
messages

In [ ]:
#importing the natural language toolkit 

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
#I will be using the vader sentiment analysis pack for this project

nltk.download('vader_lexicon')

In [ ]:
sentiment_analysis = SentimentIntensityAnalyzer()

In [ ]:
#testing the sentiment analyser

sentiment_analysis.polarity_scores('i have no opinion on james')

In [ ]:
#I will be using the compound results

def polarity(text):
    return sentiment_analysis.polarity_scores(text)['compound']

This was an attempt to run the sentiment analysis on my text, however I ran into a recursion error, I left in my commented code to document my own learning.

In [ ]:
#import nltk
#import re

#def letters_only(text):
    #re.sub("[^a-zA-Z]",  # Search for all non-letters
                          #" ",          # Replace all non-letters with spaces
                          #str(messages))
    #return letters_only(text)
#messages['letters only'] = messages['content'].apply(letters_only)

In [ ]:
#import sys
#sys.setrecursionlimit(4000)


#messages['letters_only'] = messages['content'].apply(letters_only)

In [ ]:
# coding: UTF-8
#try:
    #title_trans=''.join(chr(c) if chr(c).isupper() or chr(c).islower() else '' for c in range(256))
#except Exception:
    #pass

In [ ]:
#try:
    #messages['letters_only'] = messages['content'].apply(title_trans)
#except Exception:
    #pass

In [ ]:
#messages

In [ ]:
#messages['result'] = messages['content'].map(lambda x: x.lstrip('+-').rstrip('aAbBcC'))

In [ ]:
#messages['no numbers'] = messages['content'].astype(str).str.replace('\D+', '')


In [ ]:
#messages

In [ ]:
#Cleaning the data by removing digits

messages['no numbers'] = messages['content'].astype(str).str.replace('\d+', '')

In [ ]:
messages

In [ ]:
#applying the sentiment 

messages['sentiment'] = messages['no numbers'].apply(polarity)

In [ ]:
messages

In [ ]:
messages.loc[9, 'content']

In [ ]:
#Finding mean results by sender name 

messages.groupby('sender_name').mean()

In [ ]:
year_month = messages.groupby(['month', 'year', 'sender_name']).mean().reset_index()

In [ ]:
#Finding mean results by month

year_month

In [ ]:
#Finding sentiment by month
Dan = year_month[year_month['sender_name'] == 'Daniel Howes']['sentiment'].values

In [ ]:
Dan

In [ ]:
Mirren = year_month[year_month['sender_name'] == 'Mirren Wilson']['sentiment'].values

In [ ]:
Mirren

In [ ]:
#Plotting the sentiment change over month in matplotlib

import matplotlib.pyplot as plt 

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(Dan,label = 'Dan')
plt.plot(Mirren,label = 'Mirren')
plt.legend()

In [ ]:
#further looking into means

date = messages.groupby(['date', 'month', 'sender_name']).mean().reset_index()

In [ ]:
date

In [ ]:
#Attempting to plot sentiment over time 

Dan = date[date['sender_name'] == 'Daniel Howes']['sentiment'].values

In [ ]:
Dan

In [ ]:
Mirren = date[date['sender_name'] == 'Mirren Wilson']['sentiment'].values

In [ ]:
Mirren

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(Dan,label = 'Dan')
plt.plot(Mirren,label = 'Mirren')

In [ ]:
#Plotting the sentiment per day

day = messages.groupby(['day', 'month', 'sender_name']).mean().reset_index()

In [ ]:
Dan = day[day['sender_name'] == 'Daniel Howes']['sentiment'].values

In [ ]:
Mirren = day[day['sender_name'] == 'Mirren Wilson']['sentiment'].values

In [ ]:
import matplotlib  
import numpy
matplotlib.rc('xtick', labelsize=20)     
matplotlib.rc('ytick', labelsize=20)


plt.figure(figsize = (20,10))
plt.plot(Dan,label='Dan',linewidth=7.0)
plt.plot(Mirren,label='Mirren', color='purple',linewidth=7.0)
plt.legend()



In [ ]:
#results = set()
#messages['no numbers'].str.lower().str.split().apply(results.update)

In [ ]:
#looking at the most common words 

from collections import Counter
variable2 = Counter(" ".join(messages['no numbers'].values.tolist()).split(" ")).items()
variable2

In [ ]:
#tokenizing the messages

import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

In [ ]:
messages['yoyo'] = messages['no numbers'].apply(tokenize)

In [ ]:
messages['wordcount'] = messages['yoyo'].apply(lambda x: len(x) - x.count(" "))

In [ ]:
messages

In [ ]:
messages.groupby('sender_name').mean()

In [ ]:
#Finding the total number of messages sent 

len(messages[messages['sender_name']=='Daniel Howes'])

In [ ]:
len(messages[messages['sender_name']=='Mirren Wilson'])

In [ ]:
#Attempting to plot the wordcount per day over time 

day = messages.groupby(['day', 'month', 'sender_name']).mean().reset_index()

Dan2 = day[day['sender_name'] == 'Daniel Howes']['wordcount'].values
Mirren2 = day[day['sender_name'] == 'Mirren Wilson']['wordcount'].values

In [ ]:
#Exporting the data to csv, I did some of my plotting in excel as I was getting frustrated with matplotlib and hadn't found seaborn and plotly yet.

export_csv = messages.to_csv (r'export_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
export_csv

In [ ]:
import matplotlib     
matplotlib.rc('xtick', labelsize=20)     
matplotlib.rc('ytick', labelsize=20)


plt.figure(figsize = (20,10))
plt.plot(Dan2,label='Dan2')
plt.plot(Mirren2,label='Mirren2')
plt.legend()


In [ ]:
#Plotting the wordcount per message per person over time

date = messages.groupby(['date', 'day', 'sender_name']).mean().reset_index()

Dan2 = date[date['sender_name'] == 'Daniel Howes']['wordcount'].values
Mirren2 = date[date['sender_name'] == 'Mirren Wilson']['wordcount'].values



In [ ]:
import matplotlib     
matplotlib.rc('xtick', labelsize=20, color = 'white')     
matplotlib.rc('ytick', labelsize=20, color = 'white')


plt.figure(figsize = (20,10))
plt.plot(Dan2,label='Dan')
plt.plot(Mirren2,label='Mirren', color='purple')
plt.legend()


Attempting to plot more stuff in matplotlib before switching to excel

In [ ]:
#messages2 = messages.groupby(['date2','sender_name', 'wordcount']).sum()
#messages2

In [ ]:
#sum (wordcount)

#Dan3 = messages2[messages2['sender_name'] == 'Daniel Howes']['sentiment'].values
#Dan = messages2[messages2['sender_name'] == 'Daniel Howes']['sentiment'].values
#print (messages.dropna(axis=1, how='all'))

#Dan3 = messages2[messages2['sender_name'] == 'Daniel Howes']['wordcount'].values

#Mirren3 = messages2[messages2['sender_name'] == 'Mirren Wilson']['wordcount'].values



In [ ]:
#Dan3 = year_todate[year_todate['sender_name'] == 'Daniel Howes']['wordcount'].values

In [ ]:
#Mirren3 = year_todate[year_todate['sender_name']=='Mirren Wilson']['wordcount'].values

In [ ]:
#import matplotlib     
#matplotlib.rc('xtick', labelsize=20, color = 'black')     
#matplotlib.rc('ytick', labelsize=20, color = 'black')


#plt.figure(figsize = (20,10))
#plt.plot(Dan3,label='Dan')
#plt.plot(Mirren3,label='Mirren', color='black')
#plt.legend()

In [ ]:
#day = messages.groupby(['day', 'month', 'sender_name']).mean().reset_index()

In [ ]:
#Dan2 = day[day['sender_name'] == 'Daniel Howes']['wordcount'].values
#Mirren2 = day[day['sender_name'] == 'Mirren Wilson']['wordcount'].values